In [1]:
!pip install -U langchain==0.0.299
!pip install -U azure-search-documents==11.4.0b8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: anyio
    Found existing installation: anyio 4.0.0
    Uninstalling anyio-4.0.0:
      Successfully uninstalled anyio-4.0.0
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.187
    Uninstalling langchain-0.0.187:
      Successfully uninstalled langchain-0.0.187

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.6/305.6 kB 3.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: azure-search-documents
    Found existing installation: azure-search-documents 11.4.0b3
    Uninstalling azure-search-documents-11.4.0b3:
      Successfully uninstalled azure-search-documents-11.4.0b3

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, ru

In [85]:
import openai
import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import AzureChatOpenAI

from langchain.vectorstores.azuresearch import AzureSearch

from langchain.chains import RetrievalQA


In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [9]:
from azure.search.documents.indexes.models import (
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    ScoringProfile,
    TextWeights,
)

In [4]:
api_type = "azure"
api_base_url = "https://openaidemonubiral.openai.azure.com/"
api_version = "2023-03-15-preview"
azure_api_key = "ff5c606c134e4d1dae3426a412df834a"

openai.api_type = api_type
openai.api_base = api_base_url
openai.api_version = api_version
openai.api_key = azure_api_key

os.environ["OPENAI_API_BASE"] = api_base_url
os.environ["OPENAI_API_KEY"] = azure_api_key

In [5]:
embeddings_azure = OpenAIEmbeddings(deployment_id="text-embedding-ada-002", chunk_size=1)

llm_azure_chatgpt = AzureChatOpenAI(
    deployment_name="nubiral-lab-01", 
    temperature=0, 
    openai_api_version=api_version)

/usr/local/lib/python3.10/site-packages/langchain/embeddings/openai.py:217: UserWarning: WARNING! deployment_id is not default parameter.
                    deployment_id was transferred to model_kwargs.
                    Please confirm that deployment_id is what you intended.
  warnings.warn(


In [6]:
test_embedding = embeddings_azure.embed_query('test de emmbeding')
len(test_embedding)

1536

# TEST BASE

In [6]:
vector_store_address: str = "https://cognitive-searchdemo.search.windows.net"
vector_store_password: str = "Ndqr05h9fitPrfEh9GxNBb2vsvelw9oUgIew1VpUvYAzSeD2n9Zi"

In [8]:
index_name: str = "vector-demo-index"

In [9]:
vector_store_cognitive_search: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings_azure.embed_query,
)

In [12]:
loader = PyPDFLoader("../Data/Implementación de aprendizaje por refuerzo profundo para el control de sistemas dinámicos no lineales.pdf")

chunk_size = 1024
chunk_overlap = 200

text_splitter = RecursiveCharacterTextSplitter(        
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
    length_function = len,
)

pages = loader.load_and_split(text_splitter)

In [14]:
len(pages)

21

In [15]:
vector_store_cognitive_search.add_documents(documents=pages)

['YTUzODExMTAtMjQwNy00OThhLTg2MDEtNDVjOTQxMzRhMzUx',
 'YzkzYmUxZTAtZjdlMC00MmE2LTk5ZWItMWY5MDg2OGI5ZWNi',
 'N2Q1ZGE1MDctYWM1YS00MGZhLWI1MTEtNGExZjczODZhZTdk',
 'MDZjYTdkYTItZWIwNS00ZDlhLTkzZWMtMDBjM2VjNzNhY2Y4',
 'NmM4NzQ0MjAtMjljYS00MDJiLTg4OTktN2IwNThiMjg0NzYy',
 'MjI1M2Y0ZTAtYTQ0YS00ODcxLWJhNmItMTA1ZTY3MjgwMzAy',
 'Yjg1YTkxMjEtZmRlNS00NjM0LTliMzUtODQ1MTdlNDFlZGM2',
 'YjcwZDEyMjgtZGMxMC00NDNiLWIzZTItN2IxNDM3OTk0ODdk',
 'Mzc0MjYwYWYtMzU5Ni00YjRhLThkZDctMjk4NTNjYzFkYmZl',
 'MzNjMmJkMGMtYzYzMi00ZDAyLWEzMjAtNGNiYzg2Y2MyOTQ3',
 'ODM5NTBkNGUtNThmMC00NDU2LTkwYjAtZDVkMDY1ZTQ2N2Yy',
 'OTRlZDk1OTAtNGJiYy00NGQ5LTk2MjQtZWYwZWY0MDRlMjkz',
 'YTZkOTE0NWItMDgwMi00NWM1LTg2NTUtY2RmY2I3NDk3NWZh',
 'ZDliY2JkNjktOTk5Ni00ZWY5LWIzZWQtM2EyNGQ0YWE0MGVj',
 'MjQwOTYxNGQtYWYxNy00ZjJjLTkyZjQtNzgzNjgzMTYyODNk',
 'N2YyZjM0NTYtNjRhMi00OWZkLWJiY2EtZGZlZTk3Y2FiZDk4',
 'NzYyOTk1YWEtYTgyZS00NmUxLWE0ZmUtODgyYmQ1NzdhNDEx',
 'YzkwMzBmZWQtYjI3Ni00ZDRlLWIwZmYtNmRhOTBkYzhlYjNm',
 'MThmOTA1ZGEtMDQwMC00OGVmLWFmODUtM2QxNWJiZTdk

In [19]:
vector_store_cognitive_search.similarity_search(
    query="quienes son los autores?",
    k=5,
    search_type="similarity",
)

[Document(page_content='controladores clásicos dada la capacidad para generalizar \nexperiencia y extraer patrones de información  de las redes \nneuronales . \n \nÍndice de Términos - Deep Q Learning, péndulo invertido, control \nno lineal, redes neuronales, inteligencia artificial.  \n \nI. INTRODUCCI ÓN \n \n esde hace  varias  décadas la  respuesta  dinámica y el control \nde sistemas basados en n-péndulos invertidos ha tenido una \ngran relevancia  en el campo de la teoría de control  [4]. Los \ncontroladores tradicionales u otros reguladores se utilizan \nampliamente debido a su simplicidad y eficiencia. Sin embargo, estos \nrequieren de un modelo matemático preciso, además presentan \nmúltiples problemas cuando los modelos presentan una alta no \nlinealidad y son inestables en el tiempo [5]. Por lo que a raíz de estos \nproblemas surge la motivación de implementar  otras técnicas  como el  \naprendizaje reforzado y aplicarlas al ámbito del control , debido a l', metadata={'sourc

In [22]:
retriever_db = vector_store_cognitive_search.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [23]:
retriever_db.get_relevant_documents("quienes son los autores?")

[Document(page_content='entorno.  \n \n \nFig. 5. Evolución de las recompensas de los modelo s provenientes del entorno, \ndichas recompensas reflejan la capacidad  en que el modelo es capaz de \nmantener el péndulo invertido estable.', metadata={'source': '../Data/Implementación de aprendizaje por refuerzo profundo para el control de sistemas dinámicos no lineales.pdf', 'page': 2}),
 Document(page_content='controladores clásicos dada la capacidad para generalizar \nexperiencia y extraer patrones de información  de las redes \nneuronales . \n \nÍndice de Términos - Deep Q Learning, péndulo invertido, control \nno lineal, redes neuronales, inteligencia artificial.  \n \nI. INTRODUCCI ÓN \n \n esde hace  varias  décadas la  respuesta  dinámica y el control \nde sistemas basados en n-péndulos invertidos ha tenido una \ngran relevancia  en el campo de la teoría de control  [4]. Los \ncontroladores tradicionales u otros reguladores se utilizan \nampliamente debido a su simplicidad y eficien

# TEST BUSQUEDA HIBRIDA

In [40]:
vector_store_address: str = "https://cognitive-searchdemo.search.windows.net"
vector_store_password: str = "Ndqr05h9fitPrfEh9GxNBb2vsvelw9oUgIew1VpUvYAzSeD2n9Zi"

In [41]:
index_name: str = "document-session-index"

In [42]:
fields = [
    SimpleField(
        name="id",
        type=SearchFieldDataType.String,
        key=True,
        filterable=True,
    ),
    SearchableField(
        name="content",
        type=SearchFieldDataType.String,
        searchable=True,
    ),
    SearchField(
        name="content_vector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=len(embeddings_azure.embed_query('test de emmbeding')),
        vector_search_configuration="default",
    ),
    SearchableField(
        name="metadata",
        type=SearchFieldDataType.String,
        searchable=True,
    ),
    # Additional field to store the session
    SearchableField(
        name="session",
        type=SearchFieldDataType.String,
        filterable=True,
    )
]

In [43]:
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings_azure.embed_query,
    fields=fields,
)

In [44]:
loader = PyPDFLoader("../Data/Implementación de aprendizaje por refuerzo profundo para el control de sistemas dinámicos no lineales.pdf")

chunk_size = 1024
chunk_overlap = 200

text_splitter = RecursiveCharacterTextSplitter(        
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
    length_function = len,
)

pages = loader.load_and_split(text_splitter)

In [45]:
def add_data_to_document_metadata(documents, field_name, field_value):
    
    for document in documents:
        
        document.metadata[field_name] = field_value
        
    return documents

In [46]:
pages = add_data_to_document_metadata(pages, 'session', 'test_001')

In [47]:
pages[1]

Document(page_content='no lineal ( péndulo invertido ) como el entorno de interacción para \nel agente de aprendiza je por refuerzo. El agente de aprendizaje \npor refuerzo elige una acción de acuerdo con el estado del sistema, \nla acción incremental puede acercarse gradualmente a la \nestrategia óptima y luego el agente actualiza la red mediante \ndiferentes recompensas obten idas en el proceso de interacción . \nPara lograr la convergencia del movimiento de un péndulo \ninvertido a su estado de equilibrio en una cantidad razonable  de \niteraciones. Los resultados indican que los algoritmos realizan \nentre 100 y 300 iteraciones en prom edio para lograr la \nconvergencia. Esta aplicación del aprendizaje automático al \ncontrol de sistemas complejos y caóticos refuerzan el enfoque de \nlos algoritmos de aprendizaje en muchos campos donde se \nrequiere lograr la autonomía del agente o sistema, fren te a los \ncontroladores clásicos dada la capacidad para generalizar \nexperiencia y ex

In [48]:
vector_store.add_documents(documents=pages)

['ZmFiM2ZjOTMtMThjNS00MDVlLWJmMmYtZTUwOTY5ZDFjN2Fk',
 'MTMxMDVmYzMtM2Y1NS00NGI3LTg3MGQtMWMzMGVhMzRjNjRh',
 'ZTRkNmFhOTItMzU4Zi00ZDhlLWExOTUtM2RjYzAzMzA5YWM3',
 'ZDM1OTI2NTEtOTgyOC00MjU0LTkxOTAtNjUzNjZmZTI2ZmM3',
 'OWIyZmIzMzAtYmM1OC00NzNmLTk5YzMtODViMWY5YzU0NTkz',
 'YWI3MjQ3NzItMTNlMi00ZDZhLWIyZWMtODBmNDgyZjljNWU1',
 'YWQxZmJmMmQtNGQ3Ni00MjVlLWIxMjUtNzUxNzNiM2I3ZmQy',
 'OGNjYjZjMGEtZDJiNC00ZDYxLWE1NDUtMjA3NDJjYTMyN2Qy',
 'ZTk4MDQ4ZDAtNGI0Yy00OWVmLTgwMmUtNmFmMTE0MmZhZGEz',
 'YjRkMGM4MDgtN2IwMi00NTg3LWFhZDYtOTAzZTFhNzE4YjFl',
 'ZDRlMzE0ZjAtNjE2NS00MjNlLTk5ZjEtNmUyOTkyYmQ1YjUy',
 'NDczZDdmNjItZjcwYy00YTc4LWFjMTctMmVhOTc0ZDc3ZTY3',
 'MjNlZDk0NGEtM2JmZC00NWY0LTg1MmItZDg1NzhiNmRiMTZk',
 'MDc4ZjIyMzQtNGI1Ny00N2UyLTllODctMjg4NTA4NjNmMGFm',
 'ODc1YmY0NDUtYmFmNy00YmMwLTgzN2YtMDVjYmY3N2U0ZTJi',
 'YmJhY2VkOGUtMDYxZi00NTljLTljYzAtMTVhOTM5NTVhYzNm',
 'NmExZTViYTMtY2ZhNC00MzQ5LWE0MTAtMmI4Mzg4YjYyNGJj',
 'MWM3MWFiN2EtZGU5MC00MDM0LWFiZjEtOTFhNmY4MTQzYmIx',
 'NGEwODFlZjktZDBlNy00ZmZkLWJhNjctMDkzNjY1MDZk

In [53]:
vector_store.similarity_search(query="Quienes son los autores?", k=5, search_type="similarity", filters="session eq 'test_001'")

[Document(page_content='3 \nLas ecuaciones  diferenciales  que describen la dinámica del sistema se \npresentan en la ecuación 3 y 4. \n \nΘ̈𝑡=𝑔sinΘ𝑡 + cos Θ𝑡 [−𝐹𝑡−𝑚𝐿Θ̇𝑡2sin Θ𝑡+𝑢𝑐sgn 𝑋̇𝑡\n𝑀+𝑚]−𝑢𝑝Θ̇𝑡\n𝑚𝐿\n𝐿 [4\n3−𝑚cos 2Θ𝑡\n𝑀+𝑚]                    (3) \n \n𝑋̈𝑡=𝐹𝑡+𝑚𝐿[Θ̇𝑡sinΘ𝑡−Θ̈𝑡cos Θ𝑡]−𝑢𝑐sgn 𝑋̇𝑡\n𝑀+𝑚                                       (4) \n \nLos rangos de los valores de la recompensa  que entrega el entono por \ncada acción se presentan  en la Tabla II.  \n \nTABLA I I \nTABLA DE RECOMPENSAS DEL ENTORNO  \nRecompensa  Estado  \n+1 Por mantener el baston ergido y dentro de los \nparametros estipulados . \n-100 Por permitir la caida del baston o que el carro se salga \nde los limites . \n \nB.  Arquitectura general  \n \n    La arquitectura está basada en un  modelo Deep Q -Learning , una red \nneuronal densa en conjunto con un buffer de memoria (Experience \nreplay ) [7], esta red neuronal se entrenó en un entorno en el cual se \nrecibe información referente a la posición del péndulo

In [56]:
vector_store_search: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings_azure.embed_query,
    fields=fields
)

In [57]:
vector_store_search.similarity_search(query="Quienes son los autores?", k=5, search_type="similarity", filters="session eq 'test_001'")

[Document(page_content='3 \nLas ecuaciones  diferenciales  que describen la dinámica del sistema se \npresentan en la ecuación 3 y 4. \n \nΘ̈𝑡=𝑔sinΘ𝑡 + cos Θ𝑡 [−𝐹𝑡−𝑚𝐿Θ̇𝑡2sin Θ𝑡+𝑢𝑐sgn 𝑋̇𝑡\n𝑀+𝑚]−𝑢𝑝Θ̇𝑡\n𝑚𝐿\n𝐿 [4\n3−𝑚cos 2Θ𝑡\n𝑀+𝑚]                    (3) \n \n𝑋̈𝑡=𝐹𝑡+𝑚𝐿[Θ̇𝑡sinΘ𝑡−Θ̈𝑡cos Θ𝑡]−𝑢𝑐sgn 𝑋̇𝑡\n𝑀+𝑚                                       (4) \n \nLos rangos de los valores de la recompensa  que entrega el entono por \ncada acción se presentan  en la Tabla II.  \n \nTABLA I I \nTABLA DE RECOMPENSAS DEL ENTORNO  \nRecompensa  Estado  \n+1 Por mantener el baston ergido y dentro de los \nparametros estipulados . \n-100 Por permitir la caida del baston o que el carro se salga \nde los limites . \n \nB.  Arquitectura general  \n \n    La arquitectura está basada en un  modelo Deep Q -Learning , una red \nneuronal densa en conjunto con un buffer de memoria (Experience \nreplay ) [7], esta red neuronal se entrenó en un entorno en el cual se \nrecibe información referente a la posición del péndulo

In [81]:
from langchain.schema import BaseRetriever, Document
from typing import List


class CustomRetriever_AzureCognitiveSearch_SessionFilter(BaseRetriever):
    
    AzureCognitiveSearch_vectordb: AzureSearch
    session_filter: str
    k: int
    
    def __init__(self, AzureCognitiveSearch_vectordb, session_filter, k):
        
        super().__init__(AzureCognitiveSearch_vectordb=AzureCognitiveSearch_vectordb, session_filter=session_filter, k=k)
        
        
    def get_relevant_documents(self, query: str) -> List[Document]:
        
        documents =  self.AzureCognitiveSearch_vectordb.similarity_search(query=query, k=self.k, search_type="similarity", filters=f"session eq '{self.session_filter}'")
        
        return documents
    
    async def aget_relevant_documents(self, query: str) -> List[Document]:
        raise NotImplementedError

In [82]:
custom_CognitiveSearch_retriever = CustomRetriever_AzureCognitiveSearch_SessionFilter(vector_store_search, 'test_001', 5)

In [83]:
custom_CognitiveSearch_retriever.get_relevant_documents("Quienes son los autores?")

[Document(page_content='3 \nLas ecuaciones  diferenciales  que describen la dinámica del sistema se \npresentan en la ecuación 3 y 4. \n \nΘ̈𝑡=𝑔sinΘ𝑡 + cos Θ𝑡 [−𝐹𝑡−𝑚𝐿Θ̇𝑡2sin Θ𝑡+𝑢𝑐sgn 𝑋̇𝑡\n𝑀+𝑚]−𝑢𝑝Θ̇𝑡\n𝑚𝐿\n𝐿 [4\n3−𝑚cos 2Θ𝑡\n𝑀+𝑚]                    (3) \n \n𝑋̈𝑡=𝐹𝑡+𝑚𝐿[Θ̇𝑡sinΘ𝑡−Θ̈𝑡cos Θ𝑡]−𝑢𝑐sgn 𝑋̇𝑡\n𝑀+𝑚                                       (4) \n \nLos rangos de los valores de la recompensa  que entrega el entono por \ncada acción se presentan  en la Tabla II.  \n \nTABLA I I \nTABLA DE RECOMPENSAS DEL ENTORNO  \nRecompensa  Estado  \n+1 Por mantener el baston ergido y dentro de los \nparametros estipulados . \n-100 Por permitir la caida del baston o que el carro se salga \nde los limites . \n \nB.  Arquitectura general  \n \n    La arquitectura está basada en un  modelo Deep Q -Learning , una red \nneuronal densa en conjunto con un buffer de memoria (Experience \nreplay ) [7], esta red neuronal se entrenó en un entorno en el cual se \nrecibe información referente a la posición del péndulo

In [87]:
chain_type = "stuff"

qa = RetrievalQA.from_chain_type(llm=llm_azure_chatgpt, chain_type=chain_type, retriever=custom_CognitiveSearch_retriever, return_source_documents=True)

In [88]:
query = "quienes son los autores?"
result = qa({"query": query})

result['result']

'Los autores son C. H. Cristancho Toloza, J. A. Tumialán Borja, C. H. Rodríguez Garavito y D. J. Lancheros Cuestas.'